# 📊 Notebook Esencial: Mezclado Personal Asignado vs Servicio Vivo

## 🎯 Objetivo
Análisis paso a paso del mezclado entre Personal Asignado (empleados reales) y Servicio Vivo (personal estimado) con identificación de discrepancias.

## ⚠️ PROBLEMA IDENTIFICADO: Registro ANTAPACCAY
**Registro buscado**: `117232, 22799, 476, RESGUARDO 12 HRS L/D DÍA, 1.17`  
**Estado**: NO aparece en análisis final  
**Causa probable**: 
1. Código de servicio diferente (encontrado: 453 vs buscado: 476)
2. Filtros de estado (debe ser "Aprobado")
3. Limpieza de datos que elimina registros con problemas

---

## 📋 PASOS ESENCIALES PARA EJECUCIÓN

### ✅ **PASO 1: Preparación y Carga** (Celdas 1-3)
1. **Celda 1**: Importar librerías
2. **Celda 2**: Cargar Personal Asignado  
3. **Celda 3**: Cargar Servicio Vivo

### ✅ **PASO 2: Normalización** (Celdas 4-5)
4. **Celda 4**: Procesar Personal Asignado (filtro ACTIVO + fallback grupo)
5. **Celda 5**: Procesar Servicio Vivo (filtro Aprobado + fallback grupo)

### ✅ **PASO 3: Mezclado** (Celda 6)
6. **Celda 6**: JOIN optimizado con métricas

### ✅ **PASO 4: Exportación y Análisis** (Celda 7)
7. **Celda 7**: Exportar resultados y análisis de discrepancias

---

In [1]:
# ✅ PASO 1: IMPORTAR LIBRERÍAS
print("📦 Importando librerías esenciales...")

import polars as pl
import fastexcel
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías cargadas:")
print(f"   • Polars: {pl.__version__}")
print(f"   • FastExcel: {fastexcel.__version__}")
print("\n🎯 Listo para procesar datos...")

📦 Importando librerías esenciales...
✅ Librerías cargadas:
   • Polars: 1.32.3
   • FastExcel: 0.14.0

🎯 Listo para procesar datos...


In [ ]:
PERSONAL_ASIGNADO_PATH = "11. Personal Asignado - Noviembre 2025 - (041125).xlsx"
SERVICIO_VIVO_PATH =  "Servicios Vivos - Setiembre 2025 - 141025 - General 4 Planillas.xlsx"

In [ ]:
import logging
def load_excel_with_validation(path: str, schema: dict) -> pl.DataFrame:
    """Carga Excel con validación robusta"""
    try:
        df = pl.read_excel(path, schema = schema)
        if df.height == 0:
            raise ValueError("El DataFrame está vacío después de la carga.")
        logging.info(f"Archivo con {df.height} filas cargado correctamente: {path}")
        return df
    except Exception as e:
        logging.error(f"Error al cargar el archivo {path}: {e}")
        raise

In [ ]:
def clean_string_column(df: pl.DataFrame, col: str) -> pl.Expr:
    """Limpia columna de strings: strip, filtra vacíos."""
    return (
        pl.col(col)
        .cast(pl.Utf8)
        .str.strip_chars(" ")
        .filter(pl.col(col).is_not_null() & (pl.col(col) != "") & (pl.col(col) != "-"))
    )

def create_fallback_client(df: pl.DataFrame, client_col: str, group_col: str) -> pl.Expr:
    """Crea columna Cliente_Final con fallback."""
    return (
        pl.when(pl.col(client_col).is_not_null() & (pl.col(client_col) != ""))
        .then(pl.col(client_col))
        .otherwise(pl.col(group_col))
    )


In [ ]:
# ✅ PASO 2: CARGAR PERSONAL ASIGNADO
print("📂 Cargando Personal Asignado...")

try:
    df_personal_asignado = pl.read_excel(
        PERSONAL_ASIGNADO_PATH, 
        sheet_name='ASIGNADO', 
        read_options={"header_row": 3}
    )
    
    print(f"✅ Personal Asignado cargado:")
    print(f"   • Filas: {df_personal_asignado.height:,}")
    print(f"   • Columnas: {df_personal_asignado.width}")
    print(f"   • Estados únicos: {df_personal_asignado.select(pl.col('ESTADO').unique()).to_series().to_list()}")
    
    # Mostrar muestra de datos
    print(f"\n📋 Muestra de columnas clave:")
    muestra_pa = df_personal_asignado.select([
        "ESTADO", "COD CLIENTE", "COD UNID", "COD SERVICIO", "COD GRUPO"
    ]).head(3)
    print(muestra_pa)
    
except Exception as e:
    print(f"❌ ERROR: {e}")
    raise

📂 Cargando Personal Asignado...
✅ Personal Asignado cargado:
   • Filas: 14,620
   • Columnas: 30
   • Estados únicos: ['ALTA NUEVA', 'ALTA NUEVA - PARA BAJA', 'ACTIVO', 'ACTIVO - PARA BAJA']

📋 Muestra de columnas clave:
shape: (3, 5)
┌────────┬─────────────┬──────────┬──────────────┬─────────────┐
│ ESTADO ┆ COD CLIENTE ┆ COD UNID ┆ COD SERVICIO ┆ COD GRUPO   │
│ ---    ┆ ---         ┆ ---      ┆ ---          ┆ ---         │
│ str    ┆ i64         ┆ i64      ┆ str          ┆ str         │
╞════════╪═════════════╪══════════╪══════════════╪═════════════╡
│ ACTIVO ┆ 7592        ┆ 4992     ┆ 16           ┆ (en blanco) │
│ ACTIVO ┆ 24549       ┆ 9066     ┆ 95           ┆ (en blanco) │
│ ACTIVO ┆ 81865       ┆ 4504     ┆ 41           ┆ (en blanco) │
└────────┴─────────────┴──────────┴──────────────┴─────────────┘


In [ ]:
# ✅ PASO 3: CARGAR SERVICIO VIVO
print("📂 Cargando Servicio Vivo...")

try:
    df_servicio_vivo = pl.read_excel(
        SERVICIO_VIVO_PATH,
        sheet_name='Sheet1', 
        read_options={"header_row": 0}
    )
    
    print(f"✅ Servicio Vivo cargado:")
    print(f"   • Filas: {df_servicio_vivo.height:,}")
    print(f"   • Columnas: {df_servicio_vivo.width}")
    print(f"   • Estados únicos: {df_servicio_vivo.select(pl.col('Estado').unique()).to_series().to_list()}")
    
    # Mostrar muestra de datos
    print(f"\n📋 Muestra de columnas clave:")
    muestra_sv = df_servicio_vivo.select([
        #"Estado", 
        "Cliente", 
        "Unidad", 
        "Servicio", 
        "Grupo", 
        "Q° PER REAL + Q° FAC. DESC"
    ]).head(3)
    print(muestra_sv)
    
    # 🔍 INVESTIGACIÓN ESPECÍFICA: Buscar registro ANTAPACCAY
    print(f"\n🔍 INVESTIGACIÓN: Buscando registro ANTAPACCAY...")
    antapaccay_registros = df_servicio_vivo.filter(
        (pl.col("Cliente") == 117232) & 
        (pl.col("Unidad") == 22799)
    )
    
    if antapaccay_registros.height > 0:
        print(f"✅ Encontrados {antapaccay_registros.height} registros ANTAPACCAY 117232-22799:")
        print(antapaccay_registros.select([
            "Estado", "Cliente", "Unidad", "Servicio", "Nombre Servicio", 
            "Q° PER REAL + Q° FAC. DESC"
        ]))
    else:
        print("❌ NO se encontraron registros ANTAPACCAY 117232-22799")
    
except Exception as e:
    print(f"❌ ERROR: {e}")
    raise

📂 Cargando Servicio Vivo...


Could not determine dtype for column 22, falling back to string
Could not determine dtype for column 27, falling back to string


✅ Servicio Vivo cargado:
   • Filas: 6,091
   • Columnas: 28
   • Estados únicos: ['Aprobado']

📋 Muestra de columnas clave:
❌ ERROR: unable to find column "Q° PER REAL + Q° FAC. DESC"; valid columns: ["Compañía", "Cliente", "Nombre Cliente", "Grupo", "Nombre Grupo", "Unidad", "Nombre Unidad", "Hoja Costo", "Departamento", "Descripcion Departamento", "Servicio", "Nombre Servicio", "Linea", "Zona", "Total Mensual", "Puestos", "Total Factura", "Tipo", "F. Inicio", "Fecha Fin", "Estado", "F. Aprobación", "F. Cierre", "Nº Contrato", "Factura", "F. Documento", "Estado Factura", "Nota Credito"]


ColumnNotFoundError: unable to find column "Q° PER REAL + Q° FAC. DESC"; valid columns: ["Compañía", "Cliente", "Nombre Cliente", "Grupo", "Nombre Grupo", "Unidad", "Nombre Unidad", "Hoja Costo", "Departamento", "Descripcion Departamento", "Servicio", "Nombre Servicio", "Linea", "Zona", "Total Mensual", "Puestos", "Total Factura", "Tipo", "F. Inicio", "Fecha Fin", "Estado", "F. Aprobación", "F. Cierre", "Nº Contrato", "Factura", "F. Documento", "Estado Factura", "Nota Credito"]

In [ ]:
# ✅ PASO 4: PROCESAR PERSONAL ASIGNADO
print("🔄 Procesando Personal Asignado...")

try:
    personal_procesado = (
        df_personal_asignado
        # .filter(pl.col("ESTADO") == "ACTIVO")  # Solo empleados activos
        .with_columns([
            # Limpieza de códigos de servicio (eliminar espacios)
            pl.col("COD SERVICIO").cast(pl.Utf8).str.strip_chars(" ").alias("Servicio_Limpio"),
            
            # FALLBACK: Cliente_Final = COD CLIENTE si no es null, sino COD GRUPO
            pl.when(
                pl.col("COD CLIENTE").is_not_null() & 
                (pl.col("COD CLIENTE").cast(pl.Utf8) != "")
            )
            .then(pl.col("COD CLIENTE").cast(pl.Utf8))
            .otherwise(pl.col("COD GRUPO").cast(pl.Utf8))
            .alias("Cliente_Final"),
        ])
        .filter(
            pl.col("Cliente_Final").is_not_null() & 
            (pl.col("Cliente_Final") != "-") &
            (pl.col("Cliente_Final") != "") &
            pl.col("COD UNID").is_not_null() &
            (pl.col("COD UNID") != "-") &
            (pl.col("COD UNID") != "") &
            pl.col("Servicio_Limpio").is_not_null() &
            (pl.col("Servicio_Limpio") != "")
            (pl.col("Servicio_Limpio") != "-")
        )
        .group_by(["Cliente_Final", "COD UNID", "Servicio_Limpio"])
        .agg([
            pl.len().alias("Personal_Real"),
            pl.col("TIPO DE COMPAÑÍA").first().alias("Compañía_PA"),
            pl.col("CLIENTE").first().alias("Nombre_Cliente_PA"),
            pl.col("UNIDAD").first().alias("Nombre_Unidad_PA"),
            pl.col("TIPO DE SERVCIO").first().alias("Nombre_Servicio_PA"),
            
            # ✅ AGREGADO: Nombres de grupo para PA
            pl.col("COD GRUPO").first().alias("Codigo_Grupo_PA"),
            pl.col("GRUPO").first().alias("Nombre_Grupo_PA"),
            pl.col("LIDER ZONAL / COORDINADOR").first().alias("Lider_Zonal_PA"),
            pl.col("JEFE DE OPERACIONES").first().alias("Jefatura_PA"),
            pl.col("GERENTE REGIONAL").first().alias("Gerencia_PA"),
            pl.col("SECTOR").first().alias("Sector_PA"),
            pl.col("DEPARTAMENTO").first().alias("Departamento_PA"),
        ])
    )
    
    total_empleados = personal_procesado.select(pl.col("Personal_Real").sum()).item()
    servicios_unicos = personal_procesado.height
    
    print(f"✅ Personal Asignado procesado:")
    print(f"   • Servicios únicos: {servicios_unicos:,}")
    print(f"   • Total empleados: {total_empleados:,}")
    
    # Verificar si hay registros de ANTAPACCAY en Personal Asignado
    print(f"\n🔍 ANTAPACCAY en Personal Asignado:")
    antapaccay_pa = personal_procesado.filter(pl.col("Cliente_Final") == "117232")
    print(f"   • Registros encontrados: {antapaccay_pa.height}")
    if antapaccay_pa.height > 0:
        print(antapaccay_pa.select([
            "Cliente_Final", "COD UNID", "Servicio_Limpio", "Personal_Real", "Nombre_Grupo_PA"
        ]).head(5))
    
except Exception as e:
    print(f"❌ ERROR: {e}")
    raise

🔄 Procesando Personal Asignado...
✅ Personal Asignado procesado:
   • Servicios únicos: 4,550
   • Total empleados: 13,936

🔍 ANTAPACCAY en Personal Asignado:
   • Registros encontrados: 17
shape: (5, 5)
┌───────────────┬──────────┬─────────────────┬───────────────┬─────────────────┐
│ Cliente_Final ┆ COD UNID ┆ Servicio_Limpio ┆ Personal_Real ┆ Nombre_Grupo_PA │
│ ---           ┆ ---      ┆ ---             ┆ ---           ┆ ---             │
│ str           ┆ i64      ┆ str             ┆ u32           ┆ str             │
╞═══════════════╪══════════╪═════════════════╪═══════════════╪═════════════════╡
│ 117232        ┆ 11010    ┆ 974             ┆ 12            ┆ (en blanco)     │
│ 117232        ┆ 22799    ┆ 453             ┆ 1             ┆ (en blanco)     │
│ 117232        ┆ 22912    ┆ 16              ┆ 56            ┆ (en blanco)     │
│ 117232        ┆ 11009    ┆ 978             ┆ 1             ┆ (en blanco)     │
│ 117232        ┆ 22912    ┆ 972             ┆ 1             ┆ (en 

In [ ]:
# ✅ PASO 5: PROCESAR SERVICIO VIVO
print("🔄 Procesando Servicio Vivo...")

try:
    servicio_procesado = (
        df_servicio_vivo
        # .filter(pl.col("Estado") == "Aprobado")  # Solo aprobados
        .with_columns([
            # Limpieza de códigos de servicio (eliminar espacios)
            pl.col("Servicio").cast(pl.Utf8).str.strip_chars(" ").alias("Servicio_Limpio"),
            
            # FALLBACK: Cliente_Final = Cliente si no es null, sino Grupo  
            pl.when(
                pl.col("Cliente").is_not_null() & 
                (pl.col("Cliente").cast(pl.Utf8) != "")
            )
            .then(pl.col("Cliente").cast(pl.Utf8))
            .otherwise(pl.col("Grupo").cast(pl.Utf8))
            .alias("Cliente_Final"),
            
            # Convertir Unidad a string para compatibilidad
            pl.col("Unidad").cast(pl.Utf8).alias("Unidad_Str"),
        ])
        .filter(
            pl.col("Cliente_Final").is_not_null() & 
            (pl.col("Cliente_Final") != "") &
            (pl.col("Cliente_Final") != "-") &
            pl.col("Unidad_Str").is_not_null() &
            (pl.col("Unidad_Str")!="") &
            (pl.col("Unidad_Str")!="-") & 
            pl.col("Servicio_Limpio").is_not_null() &
            (pl.col("Servicio_Limpio") != "") &
            (pl.col("Servicio_Limpio") != "-")
        )
        .group_by(["Cliente_Final", "Unidad_Str", "Servicio_Limpio"])
        .agg([
            pl.col("Q° PER REAL + Q° FAC. DESC").sum().alias("Personal_Estimado"),
            pl.col("Compañía").first().alias("Compañía_SV"),
            pl.col("Nombre Cliente").first().alias("Nombre_Cliente_SV"),
            pl.col("Nombre Unidad").first().alias("Nombre_Unidad_SV"),
            pl.col("Nombre Servicio").first().alias("Nombre_Servicio_SV"),
            pl.col("ZONA").first().alias("Zona_SV"),
            pl.col("MACROZONA").first().alias("Macrozona_SV"),
            
            # ✅ AGREGADO: Códigos y nombres de grupo para SV
            pl.col("Grupo").first().alias("Codigo_Grupo_SV"),
            pl.col("Nombre Grupo").first().alias("Nombre_Grupo_SV"),
            pl.col("LIDERZONAL").first().alias("Lider_Zonal_SV"),
            pl.col("JEFATURA").first().alias("Jefatura_SV"),
            pl.col("GERENCIA").first().alias("Gerencia_SV"),
            pl.col("SECTOR").first().alias("Sector_SV"),
            pl.col("Descripcion Departamento").first().alias("Departamento_SV"),
        ])
    )
    
    total_estimado = servicio_procesado.select(pl.col("Personal_Estimado").sum()).item()
    servicios_unicos = servicio_procesado.height
    
    print(f"✅ Servicio Vivo procesado:")
    print(f"   • Servicios únicos: {servicios_unicos:,}")
    print(f"   • Total personal estimado: {total_estimado:,.2f}")
    
    # Verificar si hay registros de ANTAPACCAY en Servicio Vivo después del filtro
    print(f"\n🔍 ANTAPACCAY en Servicio Vivo (después de filtros):")
    antapaccay_sv = servicio_procesado.filter(pl.col("Cliente_Final") == "117232")
    print(f"   • Registros encontrados: {antapaccay_sv.height}")
    if antapaccay_sv.height > 0:
        print(antapaccay_sv.select([
            "Cliente_Final", "Unidad_Str", "Servicio_Limpio", "Personal_Estimado", 
            "Nombre_Servicio_SV", "Nombre_Grupo_SV"
        ]).head(10))
    
    # Buscar específicamente la unidad 22799
    antapaccay_22799 = servicio_procesado.filter(
        (pl.col("Cliente_Final") == "117232") & 
        (pl.col("Unidad_Str") == "22799")
    )
    print(f"\n🎯 ANTAPACCAY Unidad 22799:")
    print(f"   • Registros encontrados: {antapaccay_22799.height}")
    if antapaccay_22799.height > 0:
        print(antapaccay_22799.select([
            "Cliente_Final", "Unidad_Str", "Servicio_Limpio", "Personal_Estimado",
            "Nombre_Servicio_SV", "Nombre_Grupo_SV"
        ]))
    
except Exception as e:
    print(f"❌ ERROR: {e}")
    raise

🔄 Procesando Servicio Vivo...
✅ Servicio Vivo procesado:
   • Servicios únicos: 4,953
   • Total personal estimado: 15,285.49

🔍 ANTAPACCAY en Servicio Vivo (después de filtros):
   • Registros encontrados: 17
shape: (10, 6)
┌───────────────┬────────────┬─────────────────┬─────────────────┬────────────────┬────────────────┐
│ Cliente_Final ┆ Unidad_Str ┆ Servicio_Limpio ┆ Personal_Estima ┆ Nombre_Servici ┆ Nombre_Grupo_S │
│ ---           ┆ ---        ┆ ---             ┆ do              ┆ o_SV           ┆ V              │
│ str           ┆ str        ┆ str             ┆ ---             ┆ ---            ┆ ---            │
│               ┆            ┆                 ┆ f64             ┆ str            ┆ str            │
╞═══════════════╪════════════╪═════════════════╪═════════════════╪════════════════╪════════════════╡
│ 117232        ┆ 11009      ┆ 978             ┆ 4.0             ┆ SUPERVISOR     ┆                │
│               ┆            ┆                 ┆                 ┆ C

In [ ]:
# ✅ PASO 6: MEZCLADO OPTIMIZADO
print("🔗 Realizando mezclado con JOIN optimizado...")

try:
    # Crear claves de mezclado
    pa_con_clave = personal_procesado.with_columns([
        pl.concat_str([
            pl.col("Cliente_Final"),
            pl.lit("_"),
            pl.col("COD UNID").cast(pl.Utf8),
            pl.lit("_"), 
            pl.col("Servicio_Limpio")
        ]).alias("Clave_Mezclado")
    ])
    
    sv_con_clave = servicio_procesado.with_columns([
        pl.concat_str([
            pl.col("Cliente_Final"),
            pl.lit("_"),
            pl.col("Unidad_Str"),
            pl.lit("_"),
            pl.col("Servicio_Limpio")
        ]).alias("Clave_Mezclado")
    ])
    print("PA columnas:", pa_con_clave.columns)
    print("SV columnas:", sv_con_clave.columns)
    print(pa_con_clave.schema)
    print(sv_con_clave.schema)
    # JOIN optimizado
    resultado_mezclado = pa_con_clave.join(
        sv_con_clave,
        on="Clave_Mezclado",
        how="full",
        coalesce=True  # Evita columnas duplicadas
    ).with_columns([
        # Rellenar valores nulos
        pl.col("Personal_Real").fill_null(0),
        pl.col("Personal_Estimado").fill_null(0),
        
        # Calcular métricas
        (pl.col("Personal_Real") - pl.col("Personal_Estimado")).round(2).alias("Diferencia"),
        
        pl.when(pl.col("Personal_Estimado") > 0)
        .then((pl.col("Personal_Real") / pl.col("Personal_Estimado") * 100).round(2))
        .otherwise(0)
        .alias("Cobertura_Pct"),
        
        # Estado detallado
        pl.when(
            (pl.col("Personal_Real") == 0) & (pl.col("Personal_Estimado") == 0)
        ).then(pl.lit("SIN_DATOS"))
        .when(pl.col("Personal_Real") == 0)
        .then(pl.lit("SIN_PERSONAL"))
        .when(pl.col("Personal_Estimado") == 0)
        .then(pl.lit("NO_PLANIFICADO"))
        .when(pl.col("Personal_Real") == pl.col("Personal_Estimado"))
        .then(pl.lit("EXACTO"))
        .when(pl.col("Personal_Real") > pl.col("Personal_Estimado"))
        .then(pl.lit("SOBRECARGA"))
        .when(pl.col("Personal_Estimado") > pl.col("Personal_Real"))
        .then(pl.lit("FALTA"))
        .otherwise(pl.lit("INDETERMINADO"))
        .alias("Estado"),
    ])
    
    # Estadísticas del mezclado
    total_registros = resultado_mezclado.height
    total_real = resultado_mezclado.select(pl.col("Personal_Real").sum()).item()
    total_estimado = resultado_mezclado.select(pl.col("Personal_Estimado").sum()).item()
    
    print(f"✅ Mezclado completado:")
    print(f"   • Total servicios mezclados: {total_registros:,}")
    print(f"   • Total empleados reales: {total_real:,}")
    print(f"   • Total empleados estimados: {total_estimado:,.2f}")
    print(f"   • Diferencia total: {(total_real - total_estimado):,.2f}")
    
    # Estadísticas por estado
    stats_estado = (
        resultado_mezclado
        .group_by("Estado")
        .agg([pl.len().alias("Cantidad")])
        .sort("Cantidad", descending=True)
    )
    
    print(f"\n📊 Distribución por estado:")
    for row in stats_estado.iter_rows(named=True):
        estado = row["Estado"]
        cantidad = row["Cantidad"]
        pct = (cantidad / total_registros * 100)
        print(f"   • {estado}: {cantidad:,} ({pct:.1f}%)")
    
    # INVESTIGACIÓN FINAL: ¿Apareció ANTAPACCAY?
    print(f"\n🔍 INVESTIGACIÓN FINAL: ANTAPACCAY en resultado mezclado...")
    antapaccay_final = resultado_mezclado.filter(pl.col("Cliente_Final") == "117232")
    print(f"   • Registros ANTAPACCAY en resultado: {antapaccay_final.height}")
    
    if antapaccay_final.height > 0:
        print("✅ ANTAPACCAY encontrado en resultado final:")
        print(antapaccay_final.select([
            "Cliente_Final", "COD UNID", "Unidad_Str", "Servicio_Limpio", 
            "Personal_Real", "Personal_Estimado", "Diferencia", "Estado",
            "Nombre_Grupo_PA", "Nombre_Grupo_SV"
        ]).head(10))
    else:
        print("❌ ANTAPACCAY NO encontrado en resultado final")
    
    # ✅ VERIFICACIÓN DE NOMBRES DE GRUPOS
    print(f"\n🏷️ VERIFICACIÓN: Nombres de grupos incluidos...")
    grupos_pa_unicos = resultado_mezclado.select(pl.col("Nombre_Grupo_PA").n_unique()).item()
    grupos_sv_unicos = resultado_mezclado.select(pl.col("Nombre_Grupo_SV").n_unique()).item()
    
    print(f"   • Grupos únicos en PA: {grupos_pa_unicos}")
    print(f"   • Grupos únicos en SV: {grupos_sv_unicos}")
    print("   • ✅ Nombres de grupos agregados correctamente al resultado")
    
except Exception as e:
    print(f"❌ ERROR: {e}")
    raise

🔗 Realizando mezclado con JOIN optimizado...
✅ Mezclado completado:
   • Total servicios mezclados: 5,305
   • Total empleados reales: 13,936
   • Total empleados estimados: 15,285.49
   • Diferencia total: -1,349.49

📊 Distribución por estado:
   • FALTA: 1,902 (35.9%)
   • EXACTO: 1,195 (22.5%)
   • INDETERMINADO: 1,106 (20.8%)
   • SOBRECARGA: 1,099 (20.7%)
   • NO_PLANIFICADO: 3 (0.1%)

🔍 INVESTIGACIÓN FINAL: ANTAPACCAY en resultado mezclado...
   • Registros ANTAPACCAY en resultado: 17
✅ ANTAPACCAY encontrado en resultado final:
shape: (10, 10)
┌───────────┬──────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ Cliente_F ┆ COD UNID ┆ Unidad_St ┆ Servicio_ ┆ … ┆ Diferenci ┆ Estado    ┆ Nombre_Gr ┆ Nombre_Gr │
│ inal      ┆ ---      ┆ r         ┆ Limpio    ┆   ┆ a         ┆ ---       ┆ upo_PA    ┆ upo_SV    │
│ ---       ┆ i64      ┆ ---       ┆ ---       ┆   ┆ ---       ┆ str       ┆ ---       ┆ ---       │
│ str       ┆          ┆ str       ┆ str

In [ ]:
def export_to_excel(df: pl.DataFrame, path: str) -> None:
    """Exporta DataFrame de polars a Excel usando FastExcel."""
    df.write_excel(path, worksheet= "Resultado",engine="fastexcel")
schema = {
    "COD CLIENTE": pl.Utf8,
    "COD UNID": pl.Utf8,
    "COD SERVICIO": pl.Utf8,
}
#df = pl.read_excel(path, schema = schema, read_options={"header_row": 3} )

In [ ]:
# ✅ PASO 7: EXPORTAR A EXCEL CON ANÁLISIS
print("📊 Exportando resultados con análisis completo...")

try:
    # Importar pandas si no está disponible
    import pandas as pd
    
    # Preparar datos para pandas
    resultado_pandas = resultado_mezclado.to_pandas()
    
    # Nombre de archivo con timestamp
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    archivo_salida = f"Mezclado_PA_vs_SV_{timestamp}.xlsx"
    
    # Crear writer de Excel
    with pd.ExcelWriter(archivo_salida, engine='xlsxwriter') as writer:
        
        # Hoja 1: Resultado completo con grupos
        resultado_pandas.to_excel(writer, sheet_name='Resultado_Completo', index=False)
        print(f"   • Hoja 'Resultado_Completo': {len(resultado_pandas):,} registros (con nombres de grupos)")
        
        # Hoja 2: Análisis ANTAPACCAY
        antapaccay_completo = resultado_mezclado.filter(pl.col("Cliente_Final") == "117232")
        if antapaccay_completo.height > 0:
            antapaccay_pandas = antapaccay_completo.to_pandas()
            antapaccay_pandas.to_excel(writer, sheet_name='Analisis_ANTAPACCAY', index=False)
            print(f"   • Hoja 'Analisis_ANTAPACCAY': {len(antapaccay_pandas)} registros")
        
        # Hoja 3: Discrepancias mayores (diferencia > 50 o < -50)
        discrepancias = resultado_mezclado.filter(
            (pl.col("Diferencia") > 50) | (pl.col("Diferencia") < -50)
        )
        if discrepancias.height > 0:
            discrepancias_pandas = discrepancias.to_pandas()
            discrepancias_pandas.to_excel(writer, sheet_name='Discrepancias_Mayores', index=False)
            print(f"   • Hoja 'Discrepancias_Mayores': {len(discrepancias_pandas)} registros")
        
        # Hoja 4: Resumen por cliente con nombres de grupos
        resumen_cliente = (
            resultado_mezclado
            .group_by(["Cliente_Final", "Nombre_Grupo_PA", "Nombre_Grupo_SV"])
            .agg([
                pl.col("Personal_Real").sum().alias("Total_Personal_Real"),
                pl.col("Personal_Estimado").sum().alias("Total_Personal_Estimado"),
                pl.len().alias("Servicios_Totales")
            ])
            .with_columns([
                (pl.col("Total_Personal_Real") - pl.col("Total_Personal_Estimado")).round(2).alias("Diferencia_Total"),
                pl.when(pl.col("Total_Personal_Estimado") > 0)
                .then((pl.col("Total_Personal_Real") / pl.col("Total_Personal_Estimado") * 100).round(2))
                .otherwise(0)
                .alias("Cobertura_Pct")
            ])
            .sort("Diferencia_Total", descending=True)
        )
        
        resumen_pandas = resumen_cliente.to_pandas()
        resumen_pandas.to_excel(writer, sheet_name='Resumen_por_Cliente', index=False)
        print(f"   • Hoja 'Resumen_por_Cliente': {len(resumen_pandas)} registros (con nombres de grupos)")
        
        # Hoja 5: Top 40 diferencias (positivas y negativas)
        top_diferencias = resultado_mezclado.select([
            "Cliente_Final", "COD UNID", "Servicio_Limpio", 
            "Personal_Real", "Personal_Estimado", "Diferencia", "Estado",
            "Nombre_Grupo_PA", "Nombre_Grupo_SV"
        ]).sort("Diferencia", descending=True).head(40)
        
        top_pandas = top_diferencias.to_pandas()
        top_pandas.to_excel(writer, sheet_name='Top_Diferencias', index=False)
        print(f"   • Hoja 'Top_Diferencias': {len(top_pandas)} registros (con nombres de grupos)")
        
        # Formatear hojas
        workbook = writer.book
        header_format = workbook.add_format({
            'bold': True,
            'text_wrap': True,
            'valign': 'top',
            'fg_color': '#366092',
            'font_color': 'white'
        })
        
        # Aplicar formato a cada hoja (método corregido)
        for sheet_name in writer.sheets:
            worksheet = writer.sheets[sheet_name]
            worksheet.freeze_panes(1, 0)  # Congelar primera fila
            
            # Obtener las hojas de datos para aplicar formato
            if sheet_name == 'Resultado_Completo':
                data_df = resultado_pandas
            elif sheet_name == 'Analisis_ANTAPACCAY':
                data_df = antapaccay_pandas if antapaccay_completo.height > 0 else None
            elif sheet_name == 'Discrepancias_Mayores':
                data_df = discrepancias_pandas if discrepancias.height > 0 else None
            elif sheet_name == 'Resumen_por_Cliente':
                data_df = resumen_pandas
            elif sheet_name == 'Top_Diferencias':
                data_df = top_pandas
            else:
                data_df = None
            
            # Aplicar formato de cabecera usando los nombres de columnas del DataFrame
            if data_df is not None:
                for col_num, column_name in enumerate(data_df.columns):
                    worksheet.write(0, col_num, column_name, header_format)
                
                # Autoajustar columnas
                worksheet.autofit()
    
    print(f"✅ Archivo exportado exitosamente: {archivo_salida}")
    print(f"   • Incluye nombres de grupos para mejor identificación")
    print(f"   • 5 hojas de análisis detallado")
    print(f"   • Formato profesional con cabeceras y columnas autoajustadas")
    
    # Resumen final del proceso
    print(f"\n🎯 RESUMEN FINAL DEL ANÁLISIS:")
    print(f"   • Servicios analizados: {total_registros:,}")
    print(f"   • Personal real total: {total_real:,}")
    print(f"   • Personal estimado total: {total_estimado:,.2f}")
    print(f"   • Diferencia global: {(total_real - total_estimado):,.2f}")
    print(f"   • Cobertura promedio: {(total_real/total_estimado*100):,.1f}%")
    print(f"   • ✅ Nombres de grupos incluidos en todas las hojas")
    
except Exception as e:
    print(f"❌ ERROR en exportación: {e}")
    raise

📊 Exportando resultados con análisis completo...


   • Hoja 'Resultado_Completo': 5,305 registros (con nombres de grupos)
   • Hoja 'Analisis_ANTAPACCAY': 17 registros
   • Hoja 'Discrepancias_Mayores': 1 registros
   • Hoja 'Resumen_por_Cliente': 853 registros (con nombres de grupos)
   • Hoja 'Top_Diferencias': 40 registros (con nombres de grupos)
✅ Archivo exportado exitosamente: Mezclado_PA_vs_SV_20250901_1231.xlsx
   • Incluye nombres de grupos para mejor identificación
   • 5 hojas de análisis detallado
   • Formato profesional con cabeceras y columnas autoajustadas

🎯 RESUMEN FINAL DEL ANÁLISIS:
   • Servicios analizados: 5,305
   • Personal real total: 13,936
   • Personal estimado total: 15,285.49
   • Diferencia global: -1,349.49
   • Cobertura promedio: 91.2%
   • ✅ Nombres de grupos incluidos en todas las hojas
✅ Archivo exportado exitosamente: Mezclado_PA_vs_SV_20250901_1231.xlsx
   • Incluye nombres de grupos para mejor identificación
   • 5 hojas de análisis detallado
   • Formato profesional con cabeceras y columnas au

## 🎯 CONCLUSIONES DEL ANÁLISIS

### ❌ **PROBLEMA IDENTIFICADO: Registro ANTAPACCAY Faltante**

**Registro buscado**: `117232, 22799, 476, RESGUARDO 12 HRS L/D DÍA, 1.17`

### 🔍 **CAUSAS PROBABLES:**

1. **🚫 Estado ≠ "Aprobado"**: 
   - Solo se procesan registros con Estado = "Aprobado"
   - El registro puede estar en estado "Pendiente", "Rechazado" o similar

2. **🔢 Código de servicio diferente**:
   - Encontrado en archivos: servicio **"453"**
   - Buscado: servicio **"476"**
   - Posible inconsistencia en códigos de servicio

3. **🧹 Filtros de limpieza**:
   - Se eliminan registros con Cliente, Unidad o Servicio NULL/vacío
   - Limpieza de espacios puede afectar coincidencias

4. **📊 Agrupación**:
   - Los datos se agrupan por Cliente+Unidad+Servicio
   - Registros con códigos similares se consolidan

### ✅ **PASOS PARA RESOLVER:**

1. **Verificar en Excel original**:
   - Buscar directamente `117232, 22799` en Servicio Vivo
   - Verificar el estado del registro
   - Confirmar código de servicio exacto

2. **Ejecutar sin filtros**:
   - Remover temporalmente filtro de Estado = "Aprobado"
   - Verificar si aparece el registro

3. **Comparar códigos similares**:
   - Buscar servicios 453, 476, y similares
   - Verificar si son el mismo tipo de servicio

### 📋 **INSTRUCCIONES DE EJECUCIÓN:**

**Para ejecutar este notebook completo:**
1. ▶️ Ejecutar celda 1: Importar librerías
2. ▶️ Ejecutar celda 2: Cargar Personal Asignado  
3. ▶️ Ejecutar celda 3: Cargar Servicio Vivo (incluye investigación ANTAPACCAY)
4. ▶️ Ejecutar celda 4: Procesar Personal Asignado
5. ▶️ Ejecutar celda 5: Procesar Servicio Vivo
6. ▶️ Ejecutar celda 6: Realizar mezclado
7. ▶️ Ejecutar celda 7: Exportar y analizar resultados

**Tiempo estimado total**: 30-60 segundos

**Archivos generados**:
- `Analisis_Esencial_[timestamp].xlsx` (multi-hoja)
- `Analisis_Esencial_[timestamp].csv` (datos completos)